<a href="https://colab.research.google.com/github/Francimaria/Sistemas_de_Multiplos_Classificadores/blob/main/exercicio1_vs_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install  deslib

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.linear_model import Perceptron
from sklearn.ensemble import BaggingClassifier,AdaBoostClassifier,BaseEnsemble
from sklearn.linear_model import SGDClassifier
from scipy import stats

from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import accuracy_score, classification_report

from deslib.static import Oracle
import rlo #RLO from https://github.com/filipeclduarte/ensemble_learning/blob/main/rlo.py

### **Questão 1**

Selecione cinco bases de dados públicas que contenham características diferentes e, para cada base, calcule o Oracle no conjunto de teste para Bagging, Adaboost, Random Subspace (50\%) e Random Oracles, variando o número de classificadores-base ${10, 20, …, 100}$. Use o Perceptron como classificador-base e divida os fold usando o 5-fold cross-validation. Analise os resultados.

**Bases de dados públicas**

df1 - https://archive.ics.uci.edu/ml/datasets/Diabetic+Retinopathy+Debrecen+Data+Set

df2 - https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29

df3 - https://archive.ics.uci.edu/ml/datasets/banknote+authentication

df4 - https://archive.ics.uci.edu/ml/datasets/Wine+Quality

df5 - https://archive.ics.uci.edu/ml/datasets/Anuran+Calls+%28MFCCs%29





In [ ]:
file_1_name = "https://raw.githubusercontent.com/Francimaria/Sistemas_de_Multiplos_Classificadores/main/dataset/messidor_features.csv"
file_2_name = "https://raw.githubusercontent.com/Francimaria/Sistemas_de_Multiplos_Classificadores/main/dataset/breast-cancer-wisconsin.csv"
file_3_name = "https://raw.githubusercontent.com/Francimaria/Sistemas_de_Multiplos_Classificadores/main/dataset/data_banknote_authentication.csv"
file_4_name = "https://raw.githubusercontent.com/Francimaria/Sistemas_de_Multiplos_Classificadores/main/dataset/winequality-white.csv"
file_5_name = "https://raw.githubusercontent.com/Francimaria/Sistemas_de_Multiplos_Classificadores/main/dataset/Frogs_MFCCs.csv"

df1 = pd.read_csv(file_1_name, header=None,skiprows=1)
df2 = pd.read_csv(file_2_name, header=None)
df3 = pd.read_csv(file_3_name, header=None)
df4 = pd.read_csv(file_4_name, header=None,skiprows=1, sep=";")
df5 = pd.read_csv(file_5_name, header=None,skiprows=1)

**Pré-processamento**

In [ ]:
print("Shape: ", df1.shape)
print(df1[df1.columns[-1]].value_counts())

print("Shape: ", df2.shape)
print("0 for benign, 1 for malignant")
df2[df2.columns[-1]].replace([2, 4], [0, 1], inplace=True)
print(df2[df2.columns[-1]].value_counts())

# Remove ID
df2 = df2.drop([df2.columns[0]], axis=1)

print("Shape: ", df3.shape)
print(df3[df3.columns[-1]].value_counts())

print("Shape: ", df4.shape)
print("Replace classes [3,4,..,9] to [0,1,...6]")
df4[df4.columns[-1]].replace([3,4,5,6,7,8,9], [0,1,2,3,4,5,6], inplace=True)
print(df4[df4.columns[-1]].value_counts())

# Remover ID, gênero, e espécie (apenas families é usado como classe)
df5 = df5.drop([23,24,25], axis=1)

print("Replace classes [Leptodactylidae, Hylidae, Dendrobatidae, Bufonidae] to [0,1,2,3]")
print("Shape: ", df5.shape)
df5[df5.columns[-1]].replace(['Leptodactylidae', 'Hylidae', 'Dendrobatidae', 'Bufonidae'], [0,1,2,3], inplace=True)
print(df5[df5.columns[-1]].value_counts())

Shape:  (1151, 20)
1    611
0    540
Name: 19, dtype: int64
Shape:  (683, 11)
0 for benign, 1 for malignant
0    444
1    239
Name: 10, dtype: int64
Shape:  (1372, 5)
0    762
1    610
Name: 4, dtype: int64
Shape:  (4898, 12)
Replace classes [3,4,..,9] to [0,1,...6]
3    2198
2    1457
4     880
5     175
1     163
0      20
6       5
Name: 11, dtype: int64
Replace classes [Leptodactylidae, Hylidae, Dendrobatidae, Bufonidae] to [0,1,2,3]
Shape:  (7195, 23)
0    4420
1    2165
2     542
3      68
Name: 22, dtype: int64


In [ ]:
data = []
classe = []
df = [df1,df2,df3,df4,df5]

# Padronização dos dados na mesma escala
for d in df:
  x = d.drop([d.columns[-1]], axis=1)
  x = np.array(preprocessing.scale(x))  
  y = np.array(d[d.columns[-1]])
  
  data.append(x)
  classe.append(y)

K-fold e inicialização dos parâmetros

In [ ]:
#random_state: Using an int will produce the same results across different calls. 
kf = StratifiedKFold(n_splits=5, random_state=5)

n_estimators = np.arange(10, 110, 10)
print(n_estimators)

base_estimator=Perceptron()
print(base_estimator)


[ 10  20  30  40  50  60  70  80  90 100]
Perceptron(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,
           fit_intercept=True, max_iter=1000, n_iter_no_change=5, n_jobs=None,
           penalty=None, random_state=0, shuffle=True, tol=0.001,
           validation_fraction=0.1, verbose=0, warm_start=False)


**Executar o pool de classificadores**

In [ ]:
def run_ensemble(pool_classifiers, X, y, oracle="deslib"):
  score = []
  oracle_score = []

  for train_index, test_index in kf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    pool_classifiers.fit(X_train, y_train)

    if oracle == "RS":
      #oracle Random Subspace 
      y_pred = pool_classifiers.predict(X_test)    
      score.append(accuracy_score(y_test, y_pred))
      oracle_score.append(Oracle_RS(pool_classifiers, X_test, y_test))
      
    elif oracle == "RLO": 
      # Oracle from RLO      
      while True:
        try:
          pool_classifiers.fit(X_train, y_train)
          preds, y_pred,error = pool_classifiers.predict(X_test, y_test)    
          score.append(accuracy_score(y_test, np.array(y_pred.T)))   
          break
        except:
            continue
        break   
      oracle_score.append(pool_classifiers.Oracle_score(X_test, y_test))    

    else: 
      # Oracle from deslib
      y_pred = pool_classifiers.predict(X_test) 
      score.append(accuracy_score(y_test, y_pred))

      oracle = Oracle(pool_classifiers).fit(X_train, y_train)

      oracle_score.append(oracle.score(X_test, y_test))

  return score, oracle_score

**Oracle do RS**

In [ ]:
# From https://github.com/amorimlb/Doc_MCS/blob/main/Exercicio1/ex1.ipynb
def Oracle_RS(clf, X_test, y_test):
    X_test = pd.DataFrame(X_test)
    base_models = clf.estimators_
    base_models_feats = clf.estimators_features_

    base_models_preds = []
    for i in range(len(base_models)):
        #selecting only the columns used for the ith base model.
        X_test_subspace = X_test.iloc[:,base_models_feats[i]] 
        y_pred = base_models[i].predict(X_test_subspace)
        base_models_preds.append(y_pred)

    oracle_hits = []
    for i in range(len(y_test)):
        oracle_hit = 0
        for j in range(len(base_models_preds)):
            if base_models_preds[j][i] == y_test[i]:
                oracle_hit = 1
                break
        oracle_hits.append(oracle_hit)

    oracle_score = np.sum(oracle_hits)/len(oracle_hits)

    return oracle_score

**Bagging**

In [ ]:
# Datafile para salvar os resultados
df_result = pd.DataFrame(columns = ['Modelo', 'Dataset', 'n_estimators', 'Score_mean','Score_std', 'Oracle_mean', 'Oracle_std'])

#Executar para todos os conjuntos de dados
for i in range(len(data)):
  for n in n_estimators:
      clf = BaggingClassifier(base_estimator=base_estimator,
                              n_estimators=n,
                              bootstrap=True)
      
      score, oracle = run_ensemble(clf, data[i], classe[i])

      new_row = {'Modelo':"Bagging",
                 'Dataset': (i+1), 
                 'n_estimators':n, 
                 'Score_mean': "%0.2f" % (np.mean(score)),
                 'Score_std': "%0.2f" % (np.std(score)),
                 'Oracle_mean': "%0.2f" % (np.mean(oracle)),
                 'Oracle_std': "%0.2f" % (np.std(oracle))}
      df_result = df_result.append(new_row, ignore_index=True)  
df_result.to_csv("/content/bagging_result.csv", index=False)
df_result


,Modelo,Dataset,n_estimators,Score_mean,Score_std,Oracle_mean,Oracle_std
0,Bagging,1,10,0.72,0.01,0.99,0.00
1,Bagging,1,20,0.72,0.02,1.00,0.00
2,Bagging,1,30,0.71,0.02,1.00,0.00
3,Bagging,1,40,0.72,0.02,1.00,0.00
4,Bagging,1,50,0.72,0.02,1.00,0.00
5,Bagging,1,60,0.72,0.02,1.00,0.00
6,Bagging,1,70,0.72,0.02,1.00,0.00
7,Bagging,1,80,0.73,0.02,1.00,0.00
8,Bagging,1,90,0.72,0.02,1.00,0.00
9,Bagging,1,100,0.73,0.01,1.00,0.00


**AdaBoost**

In [ ]:
# Datafile para salvar os resultados
df_result = pd.DataFrame(columns = ['Modelo', 'Dataset', 'n_estimators', 'Score_mean','Score_std', 'Oracle_mean', 'Oracle_std'])

#Executar para todos os conjuntos de dados
for i in range(len(data)):
  for n in n_estimators:
    clf = AdaBoostClassifier(base_estimator=base_estimator,
                          algorithm='SAMME',
                          n_estimators=n)
    score, oracle = run_ensemble(clf, data[i], classe[i])    
    new_row = {'Modelo':"AdaBoost",
              'Dataset': (i+1), 
              'n_estimators':n, 
              'Score_mean': "%0.2f" % (np.mean(score)),
              'Score_std': "%0.2f" % (np.std(score)),
              'Oracle_mean': "%0.2f" % (np.mean(oracle)),
              'Oracle_std': "%0.2f" % (np.std(oracle))}
    df_result = df_result.append(new_row, ignore_index=True)

df_result.to_csv("/content/adaboost_result.csv", index=False)

df_result

**Random Subspace (50%)**

In [ ]:
# Datafile para salvar os resultados
df_result = pd.DataFrame(columns = ['Modelo', 'Dataset', 'n_estimators', 'Score_mean','Score_std', 'Oracle_mean', 'Oracle_std'])

#Executar para todos os conjuntos de dados
for i in range(len(data)):
  for n in n_estimators:
    clf = BaggingClassifier(base_estimator=base_estimator,
                            n_estimators=n,
                            max_features=0.5,
                            bootstrap=False)
    score, oracle = run_ensemble(clf, data[i], classe[i], oracle="RS")    
    new_row = {'Modelo':"RS",
              'Dataset': (i+1), 
              'n_estimators':n,  
              'Score_mean': "%0.2f" % (np.mean(score)),
              'Score_std': "%0.2f" % (np.std(score)),
              'Oracle_mean': "%0.2f" % (np.mean(oracle)),
              'Oracle_std': "%0.2f" % (np.std(oracle))}
    df_result = df_result.append(new_row, ignore_index=True)

df_result.to_csv("/content/rs_result.csv", index=False)
df_result

,Modelo,Dataset,n_estimators,Score_mean,Score_std,Oracle_mean,Oracle_std
0,RS,1,10,0.70,0.04,1.00,0.00
1,RS,1,20,0.71,0.03,1.00,0.00
2,RS,1,30,0.70,0.02,1.00,0.00
3,RS,1,40,0.70,0.05,1.00,0.00
4,RS,1,50,0.69,0.03,1.00,0.00
5,RS,1,60,0.73,0.03,1.00,0.00
6,RS,1,70,0.73,0.01,1.00,0.00
7,RS,1,80,0.72,0.02,1.00,0.00
8,RS,1,90,0.71,0.03,1.00,0.00
9,RS,1,100,0.71,0.02,1.00,0.00


**Random Oracles**

In [ ]:
# Datafile para salvar os resultados
df_result = pd.DataFrame(columns = ['Modelo', 'Dataset', 'n_estimators', 'Score_mean','Score_std', 'Oracle_mean', 'Oracle_std'])
#Executar para todos os conjuntos de dados
for i in range(len(data)):
  for n in n_estimators:
    clf = rlo.RLO(base_estimator=base_estimator, n_estimators=n)

    score, oracle = run_ensemble(clf, data[i], classe[i], oracle="RLO")    
    new_row = {'Modelo':"RLO",
              'Dataset': (i+1), 
              'n_estimators':n,  
              'Score_mean': "%0.2f" % (np.mean(score)),
              'Score_std': "%0.2f" % (np.std(score)),
              'Oracle_mean': "%0.2f" % (np.mean(oracle)),
              'Oracle_std': "%0.2f" % (np.std(oracle))}
    df_result = df_result.append(new_row, ignore_index=True)
df_result.to_csv("/content/rlo_result.csv", index=False)
df_result

**Plot gráficos**

In [ ]:
import seaborn as sns

#df_result = pd.DataFrame(columns = ['Modelo', 'Dataset', 'n_estimators', 'Score_mean','Score_std', 'Oracle_mean', 'Oracle_std'])
df_1  = pd.read_csv("/content/bagging_result.csv")
df_2 = pd.read_csv("/content/adaboost_result.csv")
df_3 = pd.read_csv("/content/rs_result.csv")
df_4 = pd.read_csv("/content/rlo_result.csv")

df_result = pd.concat([df_1, df_2,df_3,df_4])

datasets = [1,2,3,4,5]
scores_mean = []
scores_std = []
sns.set_theme(style="whitegrid")

for idx, val in enumerate(datasets):
    print("Dataset : ", val)
    df_data = df_result.loc[(df_result['Dataset'] == val)]
    print(np.array(df_data.Modelo))
    mean = np.array(df_data.Oracle_mean)
    std = np.array(df_data.Oracle_std)
    # for i in range(len(df_data)):
    #   print(str(mean[i]) + "(" + str(std[i]) + ")")
    fig = plt.figure(figsize=(8,5))
    ax.xmin = 10   
    ax = sns.lineplot(x="n_estimators", y="Oracle_mean",ci="Oracle_std", hue="Modelo", data=df_data)
    ax.legend(loc='center right',bbox_to_anchor=(1.1, 0.8), ncol=1, title="Modelo")
    file_name = '/content/result_' + str(val) +'.png'
    
    print(file_name)
    fig.savefig(file_name)
    plt.show()




Dataset :  1
['Bagging' 'Bagging' 'Bagging' 'Bagging' 'Bagging' 'Bagging' 'Bagging'
 'Bagging' 'Bagging' 'Bagging' 'AdaBoost' 'AdaBoost' 'AdaBoost' 'AdaBoost'
 'AdaBoost' 'AdaBoost' 'AdaBoost' 'AdaBoost' 'AdaBoost' 'AdaBoost' 'RS'
 'RS' 'RS' 'RS' 'RS' 'RS' 'RS' 'RS' 'RS' 'RS' 'RLO' 'RLO' 'RLO' 'RLO'
 'RLO' 'RLO' 'RLO' 'RLO' 'RLO' 'RLO']
0.99(0.01)
1.0(0.0)
1.0(0.0)
1.0(0.0)
1.0(0.0)
1.0(0.0)
1.0(0.0)
1.0(0.0)
1.0(0.0)
1.0(0.0)
1.0(0.0)
1.0(0.0)
1.0(0.0)
0.95(0.08)
1.0(0.0)
0.99(0.02)
0.98(0.03)
0.98(0.03)
0.99(0.01)
1.0(0.0)
1.0(0.0)
1.0(0.0)
1.0(0.0)
1.0(0.0)
1.0(0.0)
1.0(0.0)
1.0(0.0)
1.0(0.0)
1.0(0.0)
1.0(0.0)
0.66(0.03)
0.66(0.04)
0.64(0.03)
0.58(0.04)
0.6(0.06)
0.62(0.05)
0.63(0.05)
0.65(0.05)
0.62(0.07)
0.64(0.05)
/content/result_5.png
Dataset :  2
['Bagging' 'Bagging' 'Bagging' 'Bagging' 'Bagging' 'Bagging' 'Bagging'
 'Bagging' 'Bagging' 'Bagging' 'AdaBoost' 'AdaBoost' 'AdaBoost' 'AdaBoost'
 'AdaBoost' 'AdaBoost' 'AdaBoost' 'AdaBoost' 'AdaBoost' 'AdaBoost' 'RS'
 'RS' 'RS' 'RS' '

### **Questão 2**

Use as mesma bases de dados e os mesmos folds da questão anterior e, para cada base: - use o SGH para gerar o pool de classificadores no conjunto de treinamento; - calcule o Oracle do pool no conjunto de teste; - verifique quantas instâncias por classe foram incorretamente classificadas; - verifique quantos hiperplanos por classe foram gerados. Analise os resultados coletados.

In [ ]:
from sgh import SGH

**SGH**

In [39]:
def run_sgh(pool_classifiers, X, y, ds_name): 
  #https://github.com/luisfredgs/IN1165-mcs-list-01/blob/master/Train_SGH.ipynb
  score = []
  oracle_score = []
  estimators = []
  _miss = []

  for train_index, test_index in kf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    pool_classifiers.fit(X_train, y_train)

    est = pool_classifiers.n_estimators
    estimators.append(est)


    # Oracle from deslib

    oracle = Oracle(pool_classifiers).fit(X_train, y_train)

    oracle_score.append(oracle.score(X_test, y_test))

    _miss.append(summary(pool_classifiers.misclassified_data_frames,est, y))
  
  misclassified = pd.concat([_miss[0],_miss[1],_miss[2],_miss[3],_miss[4]])
  misclassified.to_latex("/content/%s_summary.tex" % str(ds_name))
  misclassified.to_csv("/content/%s_summary.csv" % str(ds_name))    

  return score, oracle_score, estimators, misclassified

In [42]:
# Datafile para salvar os resultados
df_result = pd.DataFrame(columns = ['Modelo', 'Dataset','estimators', 'Score_mean','Score_std', 'Oracle_mean', 'Oracle_std'])
#Executar para todos os conjuntos de dados
for i in range(len(data)):
  clf = SGH(base_estimator=base_estimator)   
  score, oracle, estimators, misclassified_data_frames = run_sgh(clf, data[i], classe[i], i) 
  new_row = {'Modelo':"SGH",
            'Dataset': (i+1), 
            'estimators':estimators,
            'Score_mean': "%0.5f" % (np.mean(score)),
            'Score_std': "%0.5f" % (np.std(score)),
            'Oracle_mean': "%0.5f" % (np.mean(oracle)),
            'Oracle_std': "%0.5f" % (np.std(oracle))}
  df_result = df_result.append(new_row, ignore_index=True)
  
df_result.to_csv("/content/sgh_result.csv", index=False)
df_result

,Modelo,Dataset,estimators,Score_mean,Score_std,Oracle_mean,Oracle_std
0,SGH,1,"[3, 7, 7, 7, 7]",nan,nan,0.99913,0.00173
1,SGH,2,"[3, 4, 4, 4, 4]",nan,nan,0.99854,0.00292
2,SGH,3,"[3, 3, 3, 3, 3]",nan,nan,1.00000,0.00000
3,SGH,4,"[12, 24, 24, 24, 24]",nan,nan,0.99878,0.00245
4,SGH,5,"[5, 12, 12, 12, 12]",nan,nan,0.97832,0.04336


In [ ]:
def summary(misclassified_data_frames,estimators, y):
  summary = []
  for df_index in range(len(misclassified_data_frames)):
      alloc = np.zeros((len(np.unique(y)), estimators))
      for i in range(0, alloc.shape[0]):
          for j in range(0, alloc.shape[1]):
              try:
                  alloc[i,j] = pool_classifiers.misclassified_data_frames[df_index][i,j]
              except:
                  pass
      summary.append(alloc)

  misclassified = pd.DataFrame(np.round(np.mean(summary, axis=0)))
  misclassified['labels'] = misclassified.index
  misclassified.columns = ['H_%s' % it for it in misclassified.columns[:-1]] + [misclassified.columns[-1]]
  return misclassified